# 검증 세트
테스트 세트를 사용하지 않으면 모델이 과대적합인지 과소적합인지 판단하기 어렵다. 테스트 세트를 사용하지 않고 이를 측정하는 간단한 방법은 훈련 세트를 또 나누는 것이다. 이 데이터를 **검증 세트(validation set)** 이라고 부른다.

이 방법이 너무 단순해서 이상하게 들릴 수도 있겠지만, 실제로 많이 사용하는 방법이다. 1절에서 전체 데이터 중 20%를 테스트 세트로 만들고 나머지 80%를 훈련 세트로 만들었다. 이 훈련 세트 중에서 다시 20%를 떼어 내어 검증 세트로 만든다.


1. 훈련 세트에서 모델을 훈련하고 검증 세트로 모델을 평가한다.  이런 식으로 테스트하고 싶은 매개변수를 바꿔가며 가장 좋은 모델을 고른다. 

2. 그다음 이 매개변수를 사용해 훈련세트와 검증 세트를 합쳐 전체 훈련 데이터에서 모델을 다시 훈련한다. 

3. 마지막에 테스트 세트에서 최종 점수를 평가한다. 아마도 실전에 투입했을 때 테스트 세트의 점수와 비슷한 성능을 기대할 수 있을 것이다.

In [1]:
import pandas as pd
wine = pd.read_csv('https://bit.ly/wine_csv_data')

In [3]:
data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()

In [4]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.2, random_state=42)

교차검증을 위해 훈련 세트에서 20%를 검증세트로 분리한다(val이 검증)

In [11]:
X_sub, X_val, y_sub, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

In [9]:
print(X_sub.shape, X_val.shape)

# 5197개였던 훈련 세트가 4157개로 줄고, 검증세트는 1040개가 되었음

(4157, 3) (1040, 3)


sub로 훈련 후 val로 모델 검증

In [12]:
from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier(random_state=42)
dt.fit(X_sub, y_sub)
print(dt.score(X_sub, y_sub))
print(dt.score(X_val, y_val))

0.9971133028626413
0.864423076923077


확실히 훈련세트에 과대적합. 매개변수를 바꿔서 더 좋은 모델을 찾아야 한다. 그 전에 검증 세트에 관해 알아야 할 것이 있다

### 교차 검증

검증 세트를 만드느라 훈련 세트가 줄었다. 보통 많은 데이터를 훈련에 사용할수록 좋은 모델이 만들어진다. 그렇다고 검증 세트를 너무 조금 떼어 놓으면 검증 점수가 들쭉날쭉하고 불안정할 것이다. 이럴 때 **교차 검증(cross validation)** 을 이용하면 안정적인 검증 점수를 얻고 훈련에 더 많은 데이터를 사용할 수 있다.

#### 교차 검증은 검증 세트를 떼어 내어 평가하는 과정을 여러 번 반복한다. 그 다음 이 점수를 평균하여 최종 검증 점수를 얻는다.

3-폴드 교차 검증이 뭔가요?
- 훈련 세트를 세 부분으로 나눠서 교차 검증을 수행하는 것을 3-폴드 교차 검증이라고 한다. 통칭 k-폴드 교차 검증(k-fold cross validation)이라고 하며, 훈련 세트를 몇 부분으로 나누냐에 따라 다르게 부른다. k-겹 교차 검증이라고도 부른다.(245p)

보통 5-폴드 교차 검증이나 10-폴드 교차 검증을 많이 사용한다. 이렇게 하면 데이터의 80~90%까지 훈련에 사용할 수 있다. 검증 세트가 줄어들지만 각 폴드에서 계산한 검증 점수를 평균하기 때문에 안정된 점수로 생각 할 수 있다.

사이킷런에는 **cross_validate()** 라는 교차 검증 함수가 있다. 사용법은 간단한데, 먼저 평가할 모델 객체를 첫 번째 매개변수로 전달한다. 그 다음 앞에서 처럼 직접 검증 세트를 떼어 내지 않고 훈련 세트 전체를 cross_validate() 함수에 전달한다.

사이킷런에는 cross_validate() 함수의 전신인 **cross_val_score()** 함수도 있다. 이 함수는 cross_validate() 함수의 결과 중에서 test_score 값만 반환하게 된다.

In [13]:
from sklearn.model_selection import cross_validate
scores = cross_validate(dt, X_train, y_train)
print(scores)

{'fit_time': array([0.01064014, 0.0066731 , 0.00630283, 0.00663114, 0.00690675]), 'score_time': array([0.00109696, 0.00113201, 0.00045323, 0.00054502, 0.00047016]), 'test_score': array([0.86923077, 0.84615385, 0.87680462, 0.84889317, 0.83541867])}


이 함수는 fit_time, score_time, test_score 키를 가진 딕셔너리를 반환한다. 처음 2개의 키는 각각 모델을 훈련하는 시간과 검증하는 시간을 의미한다. 각 키마다 5개의 숫자가 담겨 있다. **cross_validate()함수는 기본적으로 5-폴드 교차 검증을 수행한다. cv 매개변수에서 폴드 수를 바꿀 수도 있다.**

(훈련과 검증 시간은 코랩에서 리소스를 사용하는 상황에 따라 달라질 수 있으므로 fit_time과 score_time 세트는 출력 결과가 책과 다를 수 있다.)

교차 검증의 최종 점수는 test_score 키에 담긴 5개의 점수를 평균하여 얻을 수 있다. 이름은 test_score지만 검증 폴드의 점수이다. 혼동하지말자.

In [14]:
import numpy as np
print(np.mean(scores['test_score']))

0.855300214703487


교차 검증을 수행하면 입력한 모델에서 얻을 수 있는 최상의 검증 점수를 가늠해 볼 수 있다.

한 가지 주의할 점은 
cross_validate()는 훈련 세트를 섞어 폴드를 나누지 않는다. 앞서 우리는 train_test_split() 함수로 전체 데이터를 섞은 후 훈련 세트를 준비했기 때문에 따로 섞을 필요가 없다. 하지만 만약 교차 검증을 할 때 훈련 세트를 섞으려면 분할기 splitter를 지정해야한다.

사이킷런의 분할기는 교차 검증에서 폴드를 어떻게 나눌지 결정해 준다. 

##### cross_validate() 함수는 기본적으로 회귀 모델일 경우 KFold 분할기를 사용하고 분류 모델일 경우 타깃 클래스를 골고루 나누기 위해 StratifiedkFold를 사용한다. 즉 앞서 수행한 교차 검증은 다음 코드와 동일하다.

훈련 세트를 나누지 않은 경우(앞서 수행한 교차검증은 다음 코드와 같음)

In [17]:
from sklearn.model_selection import StratifiedKFold

scores = cross_validate(dt, X_train, y_train, cv=StratifiedKFold())
print(np.mean(scores['test_score']))

0.855300214703487


만약 훈련 세트를 섞은 후 10-폴드 교차 검증을 수행하려면 다음처럼 작성

In [19]:
splitter = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

scores = cross_validate(dt, X_train, y_train, cv=splitter)

print(np.mean(scores['test_score']))

0.8574181117533719


### 하이퍼파라미터 튜닝

먼저 라이브러리가 제공하는 기본값을 그대로 사용해 모델을 훈련한다. 그 다음 검증 세트의 점수나 교차 검증을 통해서 매개변수를 조금씩 바꿔 본다. 모델마다 적게는 1 ~ 2개에서, 많게는 5 ~ 6개의 매개변수를 제공한다. 이 매개변수를 바꿔가면서 모델을 훈련하고 교차 검즘을 수행해야 한다.

- 사람의 개입 없이 하이퍼파라미터 튜닝을 자동으로 수행하는 기술을 'AutoML'이라고 부른다.

그런데 아주 중요한 점이 있다. 가령 결정 트리 모델에서 최적의 max_depth 값을 찾았다고 가정해보자. 그다음 max_depth를 최적의 값으로 고정하고 min_samples_split을 바꿔가며 최적의 값을 찾는다. 이렇게 한 매개변수의 최적값을 찾고 다른 매개변수의 최적값을 찾아도 될까요? 아니요, 틀렸다. 불행하게도 max_depth의 최적값은 min_samples_split 매개변수의 값이 바뀌면 함께 달라진다. 즉 이 두 매개변수를 동시에 바꿔가면 최적의 값을 찾아야 하는 것이다.

게다가 매개변수가 많아지면 문제는 더 복잡해 진다. 파이썬의 for반복문으로 이런 과정을 직접 구현할 수도 있지만, 이미 만들어진 도구를 사용하는게 편리하겠다. 사이킷런에서 제공하는 그리드 서치 Grid Search를 사용하자.

##### 사이킷런의 GridSearchCV 클래스는 친절하게도 하이퍼파라미터 탐색과 교차 검증을 한 번에 수행한다. 별도로 cross_validat()함수를 호출할 필요가 없다. 
그럼 어떻게 사용하는지 간단한 예를 만들어보자. 기본 매개변수를 사용할 결정 트리 모델에서 min_impurity_decrease 매개변수의 최적값을 찾아본다. 먼저 GridSearchCV 클래스를 임포트하고 탐색할 매개변수와 탐색할 값의 리스트를 딕셔너리로 만든다.

#### 까먹을까봐
min_impurity_decrease 매개변수
- 어떤 노드의 정보이득 * (노드의샘플수 / 전체샘플수) 값이 기 매개변수보다 작으면 더이상 분할하지 않음

In [22]:
from sklearn.model_selection import GridSearchCV
params = {'min_impurity_decrease': [0.0001, 0.0002, 0.0003, 0.0004, 0.0005]}

# 탐색할 매개변수 : 탐색값

여기서는 0.0001부터 0.0005까지 0.0001씩 증가하는 5개의 값을 시도하겠다. GridSearchCV 클래스에 탐색 대상 모델과 params 변수를 전달하여 그리드 서치 객체를 만든다.

In [23]:
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)

# GridSearchCV(트리모델, 전달파라미터(dic으로(탐색할 매개변수: 탐색값),n_jobs)

결정 트리 클래스의 객체를 생성하자마자 바로 전달했다. 어렵지 않다. 

그다음 일반 모델을 훈련하는 것처럼 gs 객체에 fit()메서드를 호출한다. 이 메서드를 호출하면 그리드 서치 객체는 결정 트리 모델 min_impuruty_decrease 값을 바꿔가며 총 5번 실행한다.

##### GridSearchCV의 cv 매개변수(cross_validate)기본값은 5이다. 

##### 따라서 min_imputiry_decrease 값마다 5-폴드 교차 검증을 수행한다. 결국 5 x 5 = 25 개의 모델을 훈련한다. 

많은 모델을 훈련하기 때문에 GridSearchCV 클래스의 **n_jobs** 매개변수에서 병렬 실행에 사용할 CPU 코어 수를 지정하는 것이 좋다. 이 매개변수의 기본값은 1이다. -1로 지정하면 시스템에 있는 모든 코어를 사용한다. 그럼 그리드 서치를 수행해보자. 실행 결과는 크게 중요하지 않아 건너 뛴다.

In [24]:
gs.fit(X_train, y_train)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'min_impurity_decrease': [0.0001, 0.0002, 0.0003,
                                                   0.0004, 0.0005]})

교차 검증에서 최적의 하이퍼파라미터를 찾으면 전체 훈련 세트로 모델을 다시 만들어야 한다고 했던 것을 기억하나?

##### 아주 편리하게도 사이킷런의 그리드 서치는 훈련이 끝나면 25개의 모델 중에서 검증 점수가 가장 높은 모델의 매개변수 조합으로 전체 훈련 세트에서 자동으로 다시 모델을 훈련한다. 
이 **모델** 은 gs 객체의 best_estimator 속성에 저장되어 있다. 이 모델을 일반 결정 트리 처럼 똑같이 사용 할 수 있다.

In [28]:
dt = gs.best_estimator_
print(dt.score(X_train, y_train))

0.9615162593804117


In [30]:
print(gs.best_params_)  # 최적의 매개변수 저장된 속성

{'min_impurity_decrease': 0.0001}


각 매개변수에서 수행한 교차 검증의 평균 점수는 cv_results_ 속성의 'mean_test_score'키에 저장되어 있다. 5번의 교차 검증으로 얻은 점수를 출력해 보자.

In [31]:
print(gs.cv_results_['mean_test_score'])

[0.86819297 0.86453617 0.86492226 0.86780891 0.86761605]


첫 번째 값이 가장 큰 것 같다. 

수동으로 고르는 것보다 넘파이 argmax() 함수를 사용하면 가장 큰 값의 인덱스를 추출할 수 있다. 

그 다음 이 인덱스를 사용해 params키에 저장된 매개변수를 출력할 수 있다. 이 값이 최상의 검증 점수를 만든 매개변수 조합이다. 
앞에서 출력한 gs.best_params_와 동일한지 확인해 보자.

In [36]:
best_index = np.argmax(gs.cv_results_['mean_test_score'])
print(gs.cv_results_['params'][best_index])

{'min_impurity_decrease': 0.0001}


In [38]:
gs.cv_results_ # 무슨 속성인지 확인해보기 위함

{'mean_fit_time': array([0.00407681, 0.00326376, 0.00441098, 0.00437055, 0.00377889]),
 'std_fit_time': array([0.00064479, 0.00018535, 0.00142902, 0.00083741, 0.00069969]),
 'mean_score_time': array([0.00048814, 0.00036526, 0.00064697, 0.00066061, 0.00048871]),
 'std_score_time': array([2.27485340e-04, 2.77947394e-05, 2.76027790e-04, 1.87530749e-04,
        1.96060654e-04]),
 'param_min_impurity_decrease': masked_array(data=[0.0001, 0.0002, 0.0003, 0.0004, 0.0005],
              mask=[False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'params': [{'min_impurity_decrease': 0.0001},
  {'min_impurity_decrease': 0.0002},
  {'min_impurity_decrease': 0.0003},
  {'min_impurity_decrease': 0.0004},
  {'min_impurity_decrease': 0.0005}],
 'split0_test_score': array([0.86923077, 0.87115385, 0.86923077, 0.86923077, 0.86538462]),
 'split1_test_score': array([0.86826923, 0.86346154, 0.85961538, 0.86346154, 0.86923077]),
 'split2_test_score': array([0.8825794 , 0.8

#### 과정정리

1. 먼저 탐색할 매개변수를 지정한다.

2. 그다음 훈련세트에서 그리드 서치를 수행하여 최상의 평균 검증 점수가 나오는 매개변수 조합을 찾는다. 이 조합은 그리드 서치 객체에 저장된다.

3. 그리드 서치는 최상의 매개변수에서 (교차 검증에 사용한 훈련 세트가 아니라) 전체 훈련 세트를 사용해 최종 모델을 훈련한다. 이 모델도 그리드 서치 객체에 저장된다.

그럼 조금 더 복잡한 매개변수 조합을 탐색해보자. 

결정 트리에서 min_impurity_decrease 는 노드를 분할하기 위한 불순도 감소 최소량을 지정한다. 여기에다가 max_depth로 트리의 깊이를 제한하고 min_samples_split 으로 노드를 나누기 위한 최소 샘플 수도 골라보겠다.

In [39]:
params = {'min_impurity_decrease': np.arange(0.0001, 0.001, 0.0001),
         'max_depth': range(5, 20, 1),
         'min_samples_split': range(2, 100, 10)
         }

# arange() : 배열 만듦(0.0001 ~ 0.0009까지 0.0001싹 늘려가며) ->  원소 9개
# range() : 5-20까지 1씩 증가하며 15개 값 만듦,     2 - 100까지 10씩 늘려가며 10개 값 만듦

range는 range iterator 자료형을 반환하고 np.arange는 numpy array 자료형을 반환합니다. 

range는 int만, np.arrage는 실수도 가능

이 매개변수로 수행할 교차 검증 횟수는 9 * 15 * 10 = 1350개이다.

기본 5-폴드 교차 검증을 수행하므로 만들어지는 모델의 수는 1350 * 5 = 6750 개나 된다.

n_jobs = -1 로 설정하고 그리드 설치 실행

In [40]:
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)
gs.fit(X_train, y_train)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'max_depth': range(5, 20),
                         'min_impurity_decrease': array([0.0001, 0.0002, 0.0003, 0.0004, 0.0005, 0.0006, 0.0007, 0.0008,
       0.0009]),
                         'min_samples_split': range(2, 100, 10)})

In [42]:
print(gs.best_params_)

{'max_depth': 14, 'min_impurity_decrease': 0.0004, 'min_samples_split': 12}


최상의 교차 검증 점수 확인

In [47]:
print(np.max(gs.cv_results_['mean_test_score']))

0.8683865773302731


훌륭하다. GridSearchCV 클래스를 사용하니 매개변수를 일일이 바꿔가며 교차 검증을 수행하지 않고 원하는 매개변수 값을 나열하면 자동으로 교차 검증을 수행해서 최상의 매개변수를 찾을 수 있다.

그런데 아직 조금 아쉬운 점이 있다. 앞에서 탐색할 매개변수의 간격을 0.0001 혹은 1로 설정했는데, 이렇게 간격을 둔 것에 특별한 근거가 없다. 이보다 더 좁거나 넓은 간격으로 시도해 볼 수 있지않을까?

### 랜덤 서치

매개변수의 값이 수치일 때 값의 범위나 간격을 미리 정하기 어려울 수 있다. 또 너무 많은 매개변수 조건이 있어 그리드 서치 수행 시간이 오래 걸릴 수 있다. 이럴때 랜덤서치 Random Search를 사용하면 좋다.

랜덤 서치에는 매개변수 값의 목록을 전달하는 것이 아니라 매개변수를 샘플링할 수 있는 확률 분포 객체를 전달한다. 확률 분포라 하니 조금 어렵게 들릴 수 있지만 간단하고 쉽다. 먼저 싸이파이에서 2개의 확률 분포 클래스를 임포트 해보자.



싸이파이(scipy)는 어떤 라이브러라 인가?
 
- 싸이파이는 파이썬의 핵심 과학 라이브러리 중 하나이다. 적분, 보간, 선형대수, 확률 등을 포함한 수치 계산 전용 라이브러리이다.  사이킷런은 넘파이와 싸이파이 기능을 많이 사용한다.

In [48]:
from scipy.stats import uniform, randint

싸이파이의 stats 서브 패키지에 있는 uniform과 randint 클래스는 모두 주어진 범위에서 고르게 값을 뽑는다. 

이를 **'균등 분포에서 샘플링한다'라고 말한다.** randint는 정숫값을 뽑고, uniform은 실숫값을 뽑는다. 사용하는 방법은 같다. 0에서 10 사이의 범위를 갖는 randint 객체를 만들고 10개의 숫자를 샘플링해 보자.

In [53]:
rgen = randint(0, 10)
rgen.rvs(10)   

# rgen 특정 범위 랜덤 객체 만들고(범위 샘플링) rgen.rvs(10) 으로 난수 10개 생성!

array([3, 8, 0, 6, 0, 7, 5, 7, 7, 5])

10개밖에 되지 않기 때문에 고르게 샘플링되는 것 같지 않지만 샘플링 숫자를 늘리면 쉽게 확인할 수 있다 1,000개를 샘플링해서 각 숫자의 개수를 세어보겠다.

In [52]:
np.unique(rgen.rvs(1000), return_counts=True)

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]),
 array([104,  99, 110, 100, 116,  91,  91, 101, 101,  87]))

unique 메소드를 적용하면 샘플링 데이터에서 중복된 값을 제거해 출력한다.

return_counts=True 설정 시, 각 데이터 값이 몇 번 나타나는지 카운트를 세서 함께 출력한다.

 

ex) 위 코드에서 1000개의 데이터 중 0이 104번, 1이 99번 샘플링됨.

##### 개수를 늘리니 어느정도 고르게 추출

In [54]:
ugen = uniform(0, 1) 
ugen.rvs(10)

array([0.47358   , 0.39469838, 0.28618899, 0.00973349, 0.14275123,
       0.0300628 , 0.1037136 , 0.57405517, 0.39462   , 0.37711825])

In [55]:
params = {'min_impurity_decrease': uniform(0.0001, 0.001),
         'max_depth': randint(20, 50),
         'min_samples_split': randint(2, 25),
         'min_samples_leaf': randint(1,25),
         }

min_imputiry_decrease 는 0.0001에서 0.001 사이의 실숫값을 샘플링 한다.

max_depth는 20에서 50 사이의 정수, min_samples_split은 2에서 25 사이의 정수, min_samples_leaf는 1에서 25 사이의 정수를 샘플링 한다. 

샘플링 횟수는 사이킷런의 랜덤 서치 클래스인 RandomizedSearchCV의 n_iter 매개변수에 지정한다.

In [57]:
from sklearn.model_selection import RandomizedSearchCV
gs = RandomizedSearchCV(DecisionTreeClassifier(random_state=42), params, n_iter=100, n_jobs=-1, random_state=42)
gs.fit(X_train, y_train)

RandomizedSearchCV(estimator=DecisionTreeClassifier(random_state=42),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'max_depth': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x174ea2f10>,
                                        'min_impurity_decrease': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x174ea1c10>,
                                        'min_samples_leaf': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x174f5bb90>,
                                        'min_samples_split': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x174ea11d0>},
                   random_state=42)

위 params에 정의된 매개변수 범위에서 총 100번(n_iter 매개변수)을 샘플링하여 교차 검증을 수행하고 최적의 매개변수 조합을 찾는다. 

앞서 그리드 서치보다 훨씬 교차 검증 수를 줄이면서 넓은 영역을 효과적으로 탐색할 수 있다. 결과를 확인해 보자. 먼저 최적의 매개변수 조합을 출력하겠다.



In [58]:
print(gs.best_params_)

{'max_depth': 39, 'min_impurity_decrease': 0.00034102546602601173, 'min_samples_leaf': 7, 'min_samples_split': 13}


In [59]:
print(np.max(gs.cv_results_['mean_test_score']))

0.8695428296438884


최적의 모델은 이미 전체 훈련 세트(X_train, y_train)로 훈련되어 bestestimator 속성에 저장되어있다. 이 모델을 최종 모델로 결정하고 테스트 세트의 성능을 확인해 보자.

In [60]:
dt = gs.best_estimator_
print(dt.score(X_test, y_test))

0.86


테스트 세트 점수는 검증 세트에 대한 점수보다 조금 작은 것이 일반적이다. 테스트 세트 점수가 아주 만족 스럽지는 않지만 다양한 매개변수를 테스트해서 얻은 결과임을 자랑스럽게 말할 수 있을 것 같다.

앞으로 수동으로 매개변수를 바꾸는 대신에, 그리도 서치나 랜덤 서치를 사용해야겠다.